## Notebook 4 – Tone Analyzer 
https://www.ibm.com/watson/developercloud/tone-analyzer.html 
https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/ 

Tone Analyzer service uses linguistic analysis to detect emotional and language tones in written text.

If you already have an IBM Cloud / Bluemix account login here https://console.bluemix.net/ 
If you have not registered for IBM Cloud - you will need to Register for a Free account here https://www.ibm.com/watson/developer/	

To create a TONE endpoint - https://console.bluemix.net/developer/watson/dashboard  LITE Plan for Tone is free

### Tone Analyzer Signals

A comma-separated list of tones for which the service is to return its analysis of the input. The indicated tones apply both to the full document and to individual sentences of the document. You can specify one or more of the following values:
    emotion
    language
    social
    
2016-05-19: The service can return results for the following tone IDs of the different categories:
        For the emotion category: 
            anger, 
            disgust, 
            fear, 
            joy, 
            sadness
        For the language category: 
            analytical, 
            confident, 
            tentative
        For the social category: 
            openness_big5, 
            conscientiousness_big5, 
            extraversion_big5, 
            agreeableness_big5,
            emotional_range_big5
         The service returns scores for all tones of a category, regardless of their values.

    
2017-09-21: The service can return results for the following tone IDs: anger, fear, joy, sadness, analytical, confident, and tentative. The service returns results only for tones whose scores meet a minimum threshold of 0.5.

If Tone Chat Score is used - these are the signals produced
    sad
    frustrated
    satisfied
    excited
    polite
    impolite
    sympathetic


## Install dependencies

In [39]:
#imports.... Run this each time after restarting the Kernel
#!pip install watson_developer_cloud
import watson_developer_cloud as watson
import json
from botocore.client import Config
import ibm_boto3


### Create Watson Tone Analysis service


### Add Credentials

Copy paste the following snippet to next cell, and add your own set of crdentials there:

```code
credentials_os = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': '',
}

credentials_tone = {
}

```

In [40]:
# The code was removed by DSX for sharing.

## Object storage client

In [41]:
client = ibm_boto3.client(service_name='s3', 
    ibm_api_key_id = credentials_os['IBM_API_KEY_ID'],
    ibm_auth_endpoint = credentials_os['IBM_AUTH_ENDPOINT'],
    config = Config(signature_version='oauth'),
    endpoint_url = 'https://s3-api.us-geo.objectstorage.service.networklayer.com')





### Tone

- `process_text()` goes throught the text and fetch sentences and concatenate transcript based on chunk size
- `analyze transcript()` calls tone analyzer endpoint and analyze the transcript
- `post_anlysis()` shows tones and their score


In [46]:
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(version = '2016-05-19',
                               username = credentials_tone['username'],
                               password = credentials_tone['password'])


chunk_size=30

def chunk_transcript(transcript, chunk_size):
    transcript = transcript.split(' ')
    return [ transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size) ] # chunking data
    

def process_text(text):
    transcript=''
    for sentence in json.loads(text)['results']:
        transcript = transcript + sentence['alternatives'][0]['transcript'] # concatenate sentences
    transcript = chunk_transcript(transcript, chunk_size) # chunk the transcript
    return transcript


def analyze_transcript(file_name):
    transcript = client.get_object(Bucket = credentials_os['BUCKET'], Key = file_name.split('.')[0]+'_text.json')['Body']
    transcript = transcript.read().decode("utf-8")
    tone_analysis={}
    for chunk in process_text(transcript):
        chunk = ' '.join(chunk)
        tone_analysis[chunk] = tone_analyzer.tone(chunk, content_type='text/plain')
    res=client.put_object(Bucket = credentials_os['BUCKET'], Key= file_name.split('.')[0]+'_tone.json', Body = json.dumps(tone_analysis))
    return tone_analysis

def print_tones(tones):
    for tone in tones:
        print(tone) ## note for self: update this and show table instead

def post_analysis(result):
    for chunk in result.keys():
        tone_categories = result[chunk]['document_tone']['tone_categories']
        print('\nchunk: ', chunk)
        for tone_category in tone_categories:
            print_tones(tone_category['tones']) #add table instead of prints


In [47]:
file_list = ['sample1-addresschange-positive.ogg',
             'sample2-address-negative.ogg',
             'sample3-shirt-return-weather-chitchat.ogg',
             'sample4-angryblender-sportschitchat-recovery.ogg',
             'sample5-calibration-toneandcontext.ogg',
             'jfk_1961_0525_speech_to_put_man_on_moon.ogg',
             'May 1 1969 Fred Rogers testifies before the Senate Subcommittee on Communications.ogg']


In [44]:
result = analyze_transcript(text_files[0])
post_analysis(result)


chunk:  of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it
{'tone_name': 'Anger', 'tone_id': 'anger', 'score': 0.008838}
{'tone_name': 'Disgust', 'tone_id': 'disgust', 'score': 0.003648}
{'tone_name': 'Fear', 'tone_id': 'fear', 'score': 0.015782}
{'tone_name': 'Joy', 'tone_id': 'joy', 'score': 0.490567}
{'tone_name': 'Sadness', 'tone_id': 'sadness', 'score': 0.025789}
{'tone_name': 'Analytical', 'tone_id': 'analytical', 'score': 0.0}
{'tone_name': 'Confident', 'tone_id': 'confident', 'score': 0.449032}
{'tone_name': 'Tentative', 'tone_id': 'tentative', 'score': 0.0}
{'tone_name': 'Openness', 'tone_id': 'openness_big5', 'score': 0.70824}
{'tone_name': 'Conscientiousness', 'tone_id': 'conscientiousness_big5', 'score': 0.997011}
{'tone_name': 'Extraversion', 'tone_id': 'extraversion_big5', 'score': 0.561312}
{'tone_name': 'Agreeableness', 'tone_id': 'agreeableness_big5', 'score': 0.999945}
{'tone_n

In [45]:
result = analyze_transcript(text_files[6])
post_analysis(result)

WatsonApiException: Error: No text given, Code: 400